In [2]:
using ITensorNetworks
using ITensors

# using ITensorNetworks
using Observers
using Printf: Format, format
using Printf
using TupleTools
using JLD
using YAML
using HDF5
import Random

push!(LOAD_PATH,pwd())
include("input_data.jl")
include("matrices.jl")
include("states.jl")
include("expectations.jl")
include("dvr.jl")
include("utility_funcs.jl")


Nspec, Nsites, Nbonds, Nsweep, e_cutoff, 
		SVD_error, gstart, delta_g, Ng,
        mbond, pairs, evod, angle, Estrength, 
		Nstates, output_filename, parity_symmetry_type,
		inversion_symmetry_type = get_input_data("input_quick_DMRG.yml"; default_filename="test")

Nsites = 4
###{tdvp_filename}.h5 will be where the data is stored (written to on the fly while propagating every 5th sweep by default)
###ToDo: we might want to parse the name for that file from input

#Calculate kinetic matrix and x operator#
Ttmp = kinetic(Nspec)
Xtmp = Xoperator(Nspec)
Ytmp = Yoperator(Nspec)
Uptmp = Upoperator(Nspec)
Downtmp = Downoperator(Nspec)

use_inversion_symmetry = inversion_symmetry_type == "even" || inversion_symmetry_type == "odd"
use_parity_symmetry = parity_symmetry_type == "even" || parity_symmetry_type == "odd"

if evod == "dvr"
	symmetry = trivial_symmetry
	if use_inversion_symmetry && use_parity_symmetry
		symmetry = dvr_symmetric_basis
	elseif use_inversion_symmetry
		symmetry = dvr_inversion_symmetry
	elseif use_parity_symmetry
		symmetry = dvr_rotation_symmetry
	end

	# define basis
	tmp1,tmp2,tmp3 = symmetry.(exp_dvr(Nspec))
	global T = tmp1
	global X = tmp2
	global Y = tmp3
end
if evod == "m"
    symmetry = trivial_symmetry
	if use_inversion_symmetry && use_parity_symmetry
		symmetry = x -> parity_symmetry(m_inversion_symmetry(x))
	elseif use_inversion_symmetry
		symmetry = m_inversion_symmetry
	elseif use_parity_symmetry
		symmetry = parity_symmetry
	end


	# define basis
	global T = symmetry(Ttmp)
	global X = symmetry(Xtmp)
	global Y = symmetry(Ytmp)
	global Up = symmetry(Uptmp)
	global Down = symmetry(Downtmp)
end


Nspec=size(T,1)

include("operators.jl")
include("observer.jl")

In [3]:
using ITensorNetworks
function create_Hamiltonian2(g, sites, pairs; Estrength=0,angle=0, evod="m", get_ampo=false)
	Nsites = length(ITensorNetworks.vertices(sites))

	Nsecond = zeros(Int64,(Nsites-1))

	for i=1:Nsites-1
		if pairs == "nearest"
			Nsecond[i]=i+1
		elseif pairs == "allpairs"
			Nsecond[i]=Nsites
        else
            throw(string("Pairs is: ", pairs))
        end
	end

	ampo = AutoMPO()
	for i=1:Nsites-1
		ampo += 1.0,"T",i
		for j=i+1:Nsecond[i]
			c=g/((abs(j-i))^3)
			if evod == "dvr"
				# y_iy_j#
				ampo += 1.0*c,"Y",i,"Y",j
				# 2*x_ix_j#
				ampo += -2.0*c,"X",i,"X",j
			else
				# up up
				# ampo +=-.75*c,"Up",i,"Up",j
				# # up down
				# ampo +=-.25*c,"Up",i,"Down",j
				# # down up 
				# ampo +=-.25*c,"Down",i,"Up",j
				# # down down
				# ampo +=-.75*c,"Down",i,"Down",j

                ampo += -1.0*c,"Y",i,"Y",j
				ampo += -2.0*c,"X",i,"X",j
			end
		end
		#Electric field#
		
		if !iszero(Estrength)
			ampo += -cos(angle)*Estrength,"X",i
			ampo += -sin(angle)*Estrength*fac2,"Y",i
		end
	end
	ampo += 1.0,"T",Nsites
	#Electric field#
	if !iszero(Estrength)
		ampo += -cos(angle)*Estrength,"X",Nsites
		ampo += -sin(angle)*Estrength*fac2,"Y",Nsites
	end
	# H = MPO(ampo,sites)
    H = ITensorNetworks.ttn(ampo, sites)
    if get_ampo
	    return H, ampo
    else
        return H
    end
end

create_Hamiltonian2 (generic function with 1 method)

In [3]:
using Graphs



g = path_graph(3)
# g = binary_tree(2)
sites = siteinds("PlaRotor",g; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)

Random.seed!(1234)
ψ = ITensorNetworks.random_ttn(sites)
gstart = 0.2
# psi = generate_initial_state(sites; parity_symmetry_type, inversion_symmetry_type)
H = create_Hamiltonian2(gstart, sites, pairs; Estrength=0,angle=0, evod=evod)
# psi = ITensorNetworks.TTN(ITensorNetwork(;link_space=2))
ψf = ITensorNetworks.dmrg(H, ψ; nsweeps=60)
inner(ψf', H, ψf)
# apply(H,ψ;init=ψ)

-0.050027134465198554

In [4]:
sites = siteinds("PlaRotor",3; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)
ψ = randomMPS(sites)
H = create_Hamiltonian(gstart, sites, pairs; Estrength=0,angle=0, evod=evod)
E,ψf = dmrg(H, ψ; nsweeps=60)
println(E)

After sweep 1 energy=4.48344684975962  maxlinkdim=13 maxerr=1.68E-16 time=4.669
After sweep 2 energy=2.064741919878395  maxlinkdim=13 maxerr=0.00E+00 time=0.044
After sweep 3 energy=1.5475956240283988  maxlinkdim=13 maxerr=0.00E+00 time=0.006
After sweep 4 energy=1.2248366735291714  maxlinkdim=13 maxerr=0.00E+00 time=0.006
After sweep 5 energy=0.894755067288027  maxlinkdim=13 maxerr=0.00E+00 time=0.007
After sweep 6 energy=0.5830987702231102  maxlinkdim=13 maxerr=0.00E+00 time=0.006
After sweep 7 energy=0.33758680886507814  maxlinkdim=13 maxerr=0.00E+00 time=0.006
After sweep 8 energy=0.1728388994542937  maxlinkdim=13 maxerr=0.00E+00 time=0.007
After sweep 9 energy=0.0735991164427363  maxlinkdim=13 maxerr=0.00E+00 time=0.059
After sweep 10 energy=0.01735559025439043  maxlinkdim=13 maxerr=0.00E+00 time=0.008
After sweep 11 energy=-0.013565335788492058  maxlinkdim=13 maxerr=0.00E+00 time=0.007
After sweep 12 energy=-0.030336360235951005  maxlinkdim=13 maxerr=0.00E+00 time=0.005
After swe

# TDVP Testing

In [13]:
using ITensorTDVP
sites = siteinds("PlaRotor",5; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)
H = create_Hamiltonian(gstart, sites, pairs; Estrength=0,angle=0, evod=evod)
ψ = randomMPS(sites)
ψf = ITensorTDVP.tdvp(H, -3im, ψ; nsweeps=2, cutoff=1e-9)
ψf2 = ITensorTDVP.tdvp(H, 3im, ψf; nsweeps=20, cutoff=1e-9)
println(inner(ψ,ψf2))

0.9830022278069154 + 0.00211406385699888im


In [14]:

# g = path_graph(3)
# g = binary_tree(2)
Nsites = 5
sites = siteinds("PlaRotor",Nsites; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)

Random.seed!(1234)
ψ_ = ITensorNetworks.TTN(MPS_to_ITensorNetwork(ψ))
H_ = ITensorNetworks.TTN(MPS_to_ITensorNetwork(H))
ψ_t2 = ITensorNetworks.tdvp(H_,-3im, ψ_;nsweeps=2, normalize=true, cutoff=1e-9,nsites=2)
# println(inner(ψ_t2', H, ψ_t2))
ψ_t3 = ITensorNetworks.tdvp(H_,3im, ψ_t2;nsweeps=20, normalize=true, cutoff=1e-9, nsites=2)
println(inner(ψ_t3,ψ_))

In [29]:
using Graphs: path_graph
g = path_graph(30)
# g = binary_tree(2)
sites = siteinds("PlaRotor",g; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)

Random.seed!(1234)
ψ = ITensorNetworks.random_ttn(sites)
gstart = 0.2
H = create_Hamiltonian2(gstart, sites, pairs; Estrength=0,angle=0, evod=evod)
ψ_t2 = ITensorNetworks.tdvp(H,-1im, ψ;nsweeps=2, normalize=true, cutoff=1e-9,nsites=2)
ψ_t3 = ITensorNetworks.tdvp(H,1im, ψ_t2;nsweeps=2, normalize=true, cutoff=1e-9, nsites=2)
println(inner(ψ_t3,ψ))

0.99999999803339 + 2.766568410631811e-7im


In [20]:
using Graphs: binary_tree
# g = path_graph(3)
g = binary_tree(3)
sites = siteinds("PlaRotor",g; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)

Random.seed!(1234)
ψ = ITensorNetworks.random_ttn(sites)
gstart = 0.2
H = create_Hamiltonian2(gstart, sites, pairs; Estrength=0,angle=0, evod=evod)
ψ_t2 = ITensorNetworks.tdvp(H,-1im, ψ;nsweeps=2, normalize=true, cutoff=1e-9,nsites=2)
ψ_t3 = ITensorNetworks.tdvp(H,1im, ψ_t2;nsweeps=2, normalize=true, cutoff=1e-9, nsites=2)
println(inner(ψ_t3,ψ))

0.995966703965943 - 0.0037524138587936417im


In [22]:
using ITensorUnicodePlots: @visualize
g = binary_tree(3)
sites = siteinds("PlaRotor",g; basis=evod,dim=Nspec, conserve_parity=false, conserve_inversion_symmetry=false)

Random.seed!(1234)
ψ = ITensorNetworks.random_ttn(sites)
@visualize ψ

    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀ψ4⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⢸⢣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀13⠀⠱⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠸⠀⠀1⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ2⠤⠤⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ψ7⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡠⠊⠁⢸⠀⠀⠀⠀⠉1⠒⠒⠤⠤⣀ψ1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀1⠤⠊13⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⢀⡠⠒1⠀⠀13⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡏⠉⠑⠒⠢⠤1⣀⡀⠀⠀⠀⠀⢀⠤⠊⠁⠀⠀⠀⢸⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀ψ5⠁⠀⠀⠀⠀⠀⠀⠘⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀13⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉ψ3⠁⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀13⠣⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀13⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠑1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠈⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢣⠀⠀⠀⠀⠀⠀⠀⠀ 
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

ITensorNetworks.TreeTensorNetwork{Int64} with 7 vertices:
7-element Dictionaries.Indices{Int64}
 1
 2
 3
 4
 5
 6
 7

and 6 edge(s):
1 => 2
1 => 3
2 => 4
2 => 5
3 => 6
3 => 7

with vertex data:
7-element Dictionaries.Dictionary{Int64, Any}
 1 │ ((dim=13|id=548|"PlaRotor,Site,n=1"), (dim=1|id=308|"1,2"), (dim=1|id=473|…
 2 │ ((dim=13|id=481|"PlaRotor,Site,n=2"), (dim=1|id=308|"1,2"), (dim=1|id=388|…
 3 │ ((dim=13|id=561|"PlaRotor,Site,n=3"), (dim=1|id=473|"1,3"), (dim=1|id=479|…
 4 │ ((dim=13|id=626|"PlaRotor,Site,n=4"), (dim=1|id=388|"2,4"))
 5 │ ((dim=13|id=683|"PlaRotor,Site,n=5"), (dim=1|id=861|"2,5"))
 6 │ ((dim=13|id=946|"PlaRotor,Site,n=6"), (dim=1|id=479|"3,6"))
 7 │ ((dim=13|id=408|"PlaRotor,Site,n=7"), (dim=1|id=723|"3,7"))

# TEsting

In [131]:
using ITensorNetworks.ModelHamiltonians: ModelHamiltonians
using Dictionaries: Dictionary

c = path_graph(6)

s = siteinds("S=1/2", c)
H = ITensorNetworks.ttn(ModelHamiltonians.heisenberg(c), s)
ψ = ITensorNetworks.random_ttn(s)
ψ_t2 = ITensorNetworks.tdvp(H,-300im, ψ;nsweeps=10, normalize=true, cutoff=1e-8,nsites=2)
println(inner(ψ_t2', H, ψ_t2))
# ψ_t3 = ITensorNetworks.tdvp(H,300im, ψ_t2;nsweeps=10, normalize=true, cutoff=1e-8, nsites=2)
# 1 - inner(ψ_t3,ψ)

-0.4653605434890912 + 2.949029909160572e-17im
